In [ ]:
using StarStats

In [ ]:
###testing to see if the version control for the notebook  works without output 

In [ ]:
function path_constructor(strings::Vector{String})
    DATA_FOLDER = ENV["STARSTATS_TEST_DATA_FOLDER"]
    return DATA_FOLDER*"/LMC/LMC_0.00_$(strings[1])_1.00.track.gz"
end

In [ ]:
using Printf
using BenchmarkTools
masses = [@sprintf("%.3f", x) for x in range(0.9,2.1,step=0.025)]

grid = ModelDataGrid([masses],[:logM])
load_grid(grid,path_constructor,gz_dataframe_loader_with_Teff_and_star_age_fix)
compute_distances_and_EEPs(grid)

In [ ]:
using Plots
plot()
for df in grid.dfs
    plot!(df.logTeff, df.logL)
end
plot!()
xflip!()

In [ ]:
plot(legend=false)
plot!(grid.dfs[14].logTeff, grid.dfs[14].logL)
EEPs = grid.EEPs[14,:]
for i in EEPs
    scatter!([grid.dfs[14].logTeff[i]], [grid.dfs[14].logL[i]])
end

track = grid.dfs[14]
xvals = LinRange(0,5,1000)
log_Teff = get_secondary_EEP.(xvals,Ref(track),Ref(:logTeff))
log_L = get_secondary_EEP.(xvals,Ref(track),Ref(:logL))

plot!(log_Teff, log_L)

In [ ]:
function path_constructor(strings::Vector{String})
    DATA_FOLDER = ENV["STARSTATS_TEST_DATA_FOLDER"]
    return DATA_FOLDER*"/LMC/LMC_$(strings[1])_$(strings[2])_$(strings[3]).track.gz"
end

In [ ]:
using Printf
using BenchmarkTools
masses = [@sprintf("%.3f", x) for x in range(0.9,2.1,step=0.025)]
rotation = [@sprintf("%.2f", x) for x in range(0.0,0.9,step=0.1)]
overshoot = [@sprintf("%.2f", x) for x in range(0.5,4.5,step=0.5)]
grid = ModelDataGrid([rotation,masses,overshoot],
    [:rotation,:logM,:overshoot])
#@benchmark 
load_grid(grid,path_constructor,gz_dataframe_loader_with_Teff_and_star_age_fix); 
compute_distances_and_EEPs(grid)

In [ ]:
observable_names = [:logTeff, :logL, :vrot]
observable_values = [4.51974, 4.289877, 70.7195]
observable_errors = [0.2, 0.2, 10]
# potentially parallelize this
grid_likelihood = ModelDataGridLikelihood(grid, observable_names, observable_values, observable_errors);

In [ ]:
CI = credible_interval(grid_likelihood, :logM,0.68,1_000_000)

In [ ]:
using Plots
ml= marginalized_likelihood(grid_likelihood,[:logM])
plot(grid.input_values[2], ml)
hline!([CI[2]])
vline!(CI[1])

In [ ]:
using Plots

ml= marginalized_likelihood(grid_likelihood,[:logM, :rotation])
contour(grid.input_values[2], grid.input_values[1], ml', fill=true)

In [ ]:
#for next time 
# track interpolation
# Add comments to package source code

# Turn MESA history into EEP
#   
# Interpolate EEP for single stars
# Incorporate interpolated data into likelihood computation
# Model comparisons

In [ ]:
using Plots
plot()
xvals = LinRange(0,5, 1000)
plot!(interpolate_grid_quantity.(Ref(grid),Ref([0.13,1.21,1.05]),:logTeff, xvals), interpolate_grid_quantity.(Ref(grid),Ref([0.13,1.21,1.05]),:logL, xvals))
plot!(interpolate_grid_quantity.(Ref(grid),Ref([0.89,1.21,3]),:logTeff, xvals), interpolate_grid_quantity.(Ref(grid),Ref([0.89,1.21,3]),:logL, xvals))

In [ ]:
using Turing, Distributions
@model function star_model(logTeff_obs, logTeff_err, logL_obs, logL_err, vrot_obs, vrot_err, grid)
    x ~ Uniform(0,3)
    logM ~ Uniform(0.9, 1.5)
    rotation ~ Uniform(0,0.9)
    overshoot ~ Uniform(0.5,1.5)
    logTeff = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:logTeff,x)
    logL = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:logL,x)
    vrot = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:vrot,x)

    logTeff_obs ~ Normal(logTeff, logTeff_err)
    logL_obs ~ Normal(logL, logL_err)
    vrot_obs ~ Normal(vrot, vrot_err)
    return logTeff_obs, logL_obs, vrot_obs
end

In [ ]:
## Here needs more analysis for optimization 
chain = sample(star_model(4.51974, 0.2, 4.289877, 0.2, 70.7195, 20, grid), HMCDA(0.15, 0.65), 2000;stream=false, progress=true);

In [ ]:
using StatsPlots
plot(chain)

In [ ]:
corner(chain)

In [ ]:
mutable struct thing_alt
    a :: Float64
end
mutable struct thing
    a
end

In [ ]:
thing1 = thing_alt(2)
thing2 = thing(2)

In [ ]:
function add_1_to_thing(thing)
    thing.a = thing.a+1
end

In [ ]:
using BenchmarkTools
@benchmark add_1_to_thing(thing1)

In [ ]:
@benchmark add_1_to_thing(thing2)

In [ ]:
@code_warntype add_1_to_thing(thing2)

In [ ]:
@code_warntype add_1_to_thing(thing1)

In [ ]:
@benchmark interpolate_grid_quantity(grid,[0.13,1.21,1.05],:logTeff, 1.7)

In [ ]:
@code_warntype  interpolate_grid_quantity(grid,[0.13,1.21,1.05],:logTeff, 1.7)